<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Tutorial-IllinoisGRMHD: symmetry__set_gzs_staggered_gfs.C

## Authors: Leo Werneck & Zach Etienne

<font color='red'>**This module is currently under development**</font>

## In this tutorial module we explain how `IllinoisGRMHD` handles symmetry options and the ghostzones of staggered gridfunctions. This module will likely be absorbed by another one once we finish documenting the code.

### Required and recommended citations:

* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

0. [Step 0](#src_dir): **Source directory creation**
1. [Step 1](#symmetry__set_gzs_staggered_gfs__c): **`symmetry__set_gzs_staggered_gfs.C`**
1. [Step 2](#code_validation): **Code validation**
1. [Step 3](#latex_pdf_output): **Output this notebook to $\LaTeX$-formatted PDF file**

<a id='src_dir'></a>

# Step 0: Source directory creation \[Back to [top](#toc)\]
$$\label{src_dir}$$

We will now use the [cmdline_helper.py NRPy+ module](Tutorial-Tutorial-cmdline_helper.ipynb) to create the source directory within the `IllinoisGRMHD` NRPy+ directory, if it does not exist yet.

In [1]:
# Step 0: Creation of the IllinoisGRMHD source directory
# Step 0a: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
nrpy_dir_path = os.path.join("..","..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

# Step 0b: Load up cmdline_helper and create the directory
import cmdline_helper as cmd
outdir = os.path.join("..","src")
cmd.mkdir(outdir)

<a id='symmetry__set_gzs_staggered_gfs__c'></a>

# Step 1: `symmetry__set_gzs_staggered_gfs.C` \[Back to [top](#toc)\]
$$\label{symmetry__set_gzs_staggered_gfs__c}$$

This functions sets equatorial symmetry for the ghostzones, regardless of how the gridfunctions are staggered. The overall flow of the algorithm is the following:

* Compute $dz = Z_{i,j,k+1} - Z_{i,j,k}$
* Set the $z$-offset based on the function's staggering along the $z$-direction, $z_{\rm offset} = \frac{dz}{2}Z_{\rm stagger}$, where $Z_{\rm stagger} = 0,1$
* Determine the number of ghostzones
* Looping over all ghostzones
    1. Determine the index associated with the position $(i,j,k)$
    1. Apply equatorial symmetry for gridfunctions

There are 4 different cases we consider:
    
1. ***Unstaggered*** gridfunctions that contain 1 ghostzone. Index structure: $(0,1,2,\ldots) \to \left(-dz,0,dz,\ldots\right)$
1. ***Unstaggered*** gridfunctions that contain 2 ghostzones. Index structure: $(0,1,2,3,4,\ldots) \to \left(-2dz,-dz,0,dz,2dz,\ldots\right)$
1. ***Staggered*** gridfunctions that contain 1 ghostzone. Index structure: $(0,1,2,\ldots) \to \left(-\frac{dz}{2},\frac{dz}{2},\frac{3dz}{2},\ldots\right)$
1. ***Staggered*** gridfunctions that contain 2 ghostzone. Index structure: $(0,1,2,3,\ldots) \to \left(-\frac{3dz}{2},-\frac{dz}{2},\frac{dz}{2},\frac{3dz}{2},\ldots\right)$

The equatorial symmetry implies that the functions have the same same values for $-z$ and $z$. For the 4 different cases above, we will then apply the following symmetries, respectively:

1. $f_{i,j,0} = af_{i,j,2}$
1. $f_{i,j,0} = af_{i,j,4}$; $f_{i,j,1} = af_{i,j,3}$
1. $f_{i,j,0} = af_{i,j,1}$
1. $f_{i,j,0} = af_{i,j,3}$; $f_{i,j,1} = af_{i,j,2}$

where $f$ represents any given gridfunction and the constant $a$ is either $+1$ or $-1$.

In [2]:
%%writefile $outdir/symmetry__set_gzs_staggered_gfs.C
#include "cctk.h"
#include "cctk_Parameters.h"
#include <cstdio>
#include <cstdlib>
#include "IllinoisGRMHD_headers.h"

void IllinoisGRMHD_set_symmetry_gzs_staggered(const cGH *cctkGH, const int *cctk_lsh,CCTK_REAL *X,CCTK_REAL *Y,CCTK_REAL *Z, CCTK_REAL *gridfunc,
                                              CCTK_REAL *gridfunc_syms,int stagger_x,int stagger_y,int stagger_z) {

  DECLARE_CCTK_PARAMETERS;

  if(CCTK_EQUALS(Symmetry, "equatorial")) 
    CCTK_VError(VERR_DEF_PARAMS,"Warning: Symmetry==equatorial not supported! USE AT YOUR OWN RISK. You will need to comment this error message out.");

  // No symmetries -> return.
  if(CCTK_EQUALS(Symmetry, "none")) return;

  CCTK_REAL dz = Z[CCTK_GFINDEX3D(cctkGH,0,0,1)] - Z[CCTK_GFINDEX3D(cctkGH,0,0,0)];

  CCTK_REAL z_offset = dz*0.5*stagger_z;

  int num_gzs=0;
  //FIXME: Might want to use cctk_nghostzones instead...
  while( (Z[CCTK_GFINDEX3D(cctkGH,0,0,num_gzs)]+z_offset) < -dz*0.1 && num_gzs<cctk_lsh[2]) num_gzs++;
  if(num_gzs*2>=cctk_lsh[2]) CCTK_VError(VERR_DEF_PARAMS,"ERROR in symmetry__set_gzs_staggered_gfs.C");

#pragma omp parallel for
  for(int k=0;k<num_gzs;k++) for(int j=0;j<cctk_lsh[1];j++) for(int i=0;i<cctk_lsh[0];i++) {
        int index_inside__sym_gz = CCTK_GFINDEX3D(cctkGH,i,j,k);

        /* This loop sets symmetry ghostzones, regardless of how the gridfunction is staggered.
         *
         * STAGGERED PATTERN:
         * if num_gzs==1 && stagger_z==1: 
         * z[] = {-dz/2,dz/2,3dz/2, etc} -> gridfunc[index 0] = gridfunc_syms[2]*gridfunc[index 1]
         *
         * if num_gzs==2 && stagger_z==1: 
         * z[] = {-3dz/2,-dz/2,dz/2,3dz/2 etc} 
         * -> gridfunc[index 0] = gridfunc_syms[2]*gridfunc[index 3]
         * -> gridfunc[index 1] = gridfunc_syms[2]*gridfunc[index 2]
         * .
         * .
         * .
         * -> gridfunc[i] = gridfunc_syms[2]*gridfunc[(num_gz*2-1)-i]
         *
         * UNSTAGGERED PATTERN:
         * if num_gzs==1 && stagger_z==0:
         * z[] = {-dz,0,dz, etc} -> gridfunc[index 0] = gridfunc_syms[2]*gridfunc[index 2]
         *
         * if num_gzs==2 && stagger_z==0:
         * z[] = {-2dz,-dz,0,dz,2dz, etc} -> gridfunc[index 0] = gridfunc_syms[2]*gridfunc[index 4]
         * z[] = {-2dz,-dz,0,dz,2dz, etc} -> gridfunc[index 1] = gridfunc_syms[2]*gridfunc[index 3]
         * .
         * .
         * .
         * -> gridfunc[i] = gridfunc_syms[2]*gridfunc[(num_gz*2)-i]
         *
         * OVERALL PATTERN: gridfunc[i] = gridfunc_syms[2]*gridfunc[(num_gz*2-stagger_z)-i] */

        int matching_index_outside_sym_gz = CCTK_GFINDEX3D(cctkGH,i,j,(num_gzs*2-stagger_z)-k);

        gridfunc[index_inside__sym_gz] = gridfunc_syms[2]*gridfunc[matching_index_outside_sym_gz];
      }
}



Overwriting ../src/symmetry__set_gzs_staggered_gfs.C


<a id='code_validation'></a>

# Step 2: Code validation \[Back to [top](#toc)\]
$$\label{code_validation}$$

First we download the original `IllinoisGRMHD` source code and then compare it to the source code generated by this tutorial notebook.

In [3]:
# # Verify if the code generated by this tutorial module
# # matches the original IllinoisGRMHD source code

# # First download the original IllinoisGRMHD source code
# import urllib
# from os import path

# original_IGM_file_url  = "https://bitbucket.org/zach_etienne/wvuthorns/raw/5611b2f0b17135538c9d9d17c7da062abe0401b6/IllinoisGRMHD/src/symmetry__set_gzs_staggered_gfs.C"
# original_IGM_file_name = "symmetry__set_gzs_staggered_gfs-original.C"
# original_IGM_file_path = os.path.join(IGM_src_dir_path,original_IGM_file_name)

# # Then download the original IllinoisGRMHD source code
# # We try it here in a couple of ways in an attempt to keep
# # the code more portable
# try:
#     original_IGM_file_code = urllib.request.urlopen(original_IGM_file_url).read().decode("utf-8")
#     # Write down the file the original IllinoisGRMHD source code
#     with open(original_IGM_file_path,"w") as file:
#         file.write(original_IGM_file_code)
# except:
#     try:
#         original_IGM_file_code = urllib.urlopen(original_IGM_file_url).read().decode("utf-8")
#         # Write down the file the original IllinoisGRMHD source code
#         with open(original_IGM_file_path,"w") as file:
#             file.write(original_IGM_file_code)
#     except:
#         # If all else fails, hope wget does the job
#         !wget -O $original_IGM_file_path $original_IGM_file_url

# # Perform validation
# Validation__symmetry__set_gzs_staggered_gfs__C  = !diff $original_IGM_file_path $outfile_path__symmetry__set_gzs_staggered_gfs__C

# if Validation__symmetry__set_gzs_staggered_gfs__C == []:
#     # If the validation passes, we do not need to store the original IGM source code file
#     !rm $original_IGM_file_path
#     print("Validation test for symmetry__set_gzs_staggered_gfs.C: PASSED!")
# else:
#     # If the validation fails, we keep the original IGM source code file
#     print("Validation test for symmetry__set_gzs_staggered_gfs.C: FAILED!")
#     # We also print out the difference between the code generated
#     # in this tutorial module and the original IGM source code
#     print("Diff:")
#     for diff_line in Validation__symmetry__set_gzs_staggered_gfs__C:
#         print(diff_line)

<a id='latex_pdf_output'></a>

# Step 3: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-IllinoisGRMHD__symmetry__set_gzs_staggered_gfs.pdf](Tutorial-IllinoisGRMHD__symmetry__set_gzs_staggered_gfs.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means).

In [4]:
latex_nrpy_style_path = os.path.join(nrpy_dir_path,"latex_nrpy_style.tplx")
#!jupyter nbconvert --to latex --template $latex_nrpy_style_path --log-level='WARN' Tutorial-IllinoisGRMHD__symmetry__set_gzs_staggered_gfs.ipynb
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__symmetry__set_gzs_staggered_gfs.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__symmetry__set_gzs_staggered_gfs.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__symmetry__set_gzs_staggered_gfs.tex
!rm -f Tut*.out Tut*.aux Tut*.log